## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

Импортируем библиотеки pandas, pymystem3, прочитаем файл data.csv и сохраним его в переменной data.

In [1]:
import pandas as pd
from pymystem3 import Mystem
m = Mystem()
data = pd.read_csv('/datasets/data.csv')

Посмотрим срез на котором видны выбросы, пропуски и другие проблемы этих данных.

In [2]:
data[10:20]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10,2,-4171.483647,36,высшее,0,женат / замужем,0,M,компаньон,0,113943.491460,покупка недвижимости
11,0,-792.701887,40,среднее,1,женат / замужем,0,F,сотрудник,0,77069.234271,покупка коммерческой недвижимости
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
13,0,-1846.641941,54,неоконченное высшее,2,женат / замужем,0,F,сотрудник,0,130458.228857,приобретение автомобиля
14,0,-1844.956182,56,высшее,0,гражданский брак,1,F,компаньон,1,165127.911772,покупка жилой недвижимости
15,1,-972.364419,26,среднее,1,женат / замужем,0,F,сотрудник,0,116820.904450,строительство собственной недвижимости
16,0,-1719.934226,35,среднее,1,женат / замужем,0,F,сотрудник,0,289202.704229,недвижимость
17,0,-2369.999720,33,высшее,0,гражданский брак,1,M,сотрудник,0,90410.586745,строительство недвижимости
18,0,400281.136913,53,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823.777243,на покупку подержанного автомобиля
19,0,-10038.818549,48,СРЕДНЕЕ,1,в разводе,3,F,сотрудник,0,242831.107982,на покупку своего автомобиля


Общая информация о данных таблицы data.

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Всего в таблице 12 столбцов с целочисленными, вещественными и object типами данных.

Название столбцов и описание информации, которую они содержат:
* children — количество детей в семье
* days_employed — общий трудовой стаж в днях
* dob_years — возраст клиента в годах
* education — уровень образования клиента
* education_id — идентификатор уровня образования
* family_status — семейное положение
* family_status_id — идентификатор семейного положения
* gender — пол клиента
* income_type — тип занятости
* debt — имел ли задолженность по возврату кредитов
* total_income — ежемесячный доход
* purpose — цель получения кредита

### Вывод

Данные записаны плохо. В столбце days_employed видны пропущенные значения, аномальный трудовой стаж и отрицательные значения. Столбец total_income также имеет пропуски и записан в виде чисел с плавающей точкой. Слова в education отличаются по регистру, что точно помешает в дальнейшей обработке. Это проблемы, которые видны на срезе. Перед тем, как приступать к анализу, стоит предобработать данные. |

### Шаг 2. Предобработка данных

### Обработка пропусков

Определим, в каких столбцах встречаются пропуски.

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Пропуски есть в двух столбцах. Начнем с days_employed, рассмотрим его.

##### Обработка пропусков столбца days_employed

In [5]:
data['days_employed'].head(10)

0     -8437.673028
1     -4024.803754
2     -5623.422610
3     -4124.747207
4    340266.072047
5      -926.185831
6     -2879.202052
7      -152.779569
8     -6929.865299
9     -2188.756445
Name: days_employed, dtype: float64

Days_employed содержит информацию об общем трудовом стаже в днях. Данные записаны в виде отрицательных чисел с плавающей точкой, что неудобно для восприятия и стоит изменить тип данных на целочисленный и убрать минус. Также встречается аномально большой стаж. 

Приведем числа в столбце к абсолютным, методом abs(), чтобы избавиться от минусов.

In [6]:
data['days_employed'] = abs(data['days_employed'])
data['days_employed'].head(10)

0      8437.673028
1      4024.803754
2      5623.422610
3      4124.747207
4    340266.072047
5       926.185831
6      2879.202052
7       152.779569
8      6929.865299
9      2188.756445
Name: days_employed, dtype: float64

Кажется, что некоторые значения в столбце days_employed записаны не в днях, а в часах. Такие строки выдают слишком большие значения. Отсортируем таблицу по убыванию чтобы их увидеть.

In [7]:
data.sort_values(by = ['days_employed'], ascending = False).head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
6954,0,401755.400475,56,среднее,1,вдовец / вдова,2,F,пенсионер,0,176278.441171,ремонт жилью
10006,0,401715.811749,69,высшее,0,Не женат / не замужем,4,F,пенсионер,0,57390.256908,получение образования
7664,1,401675.093434,61,среднее,1,женат / замужем,0,F,пенсионер,0,126214.519212,операции с жильем
2156,0,401674.466633,60,среднее,1,женат / замужем,0,M,пенсионер,0,325395.724541,автомобили
7794,0,401663.850046,61,среднее,1,гражданский брак,1,F,пенсионер,0,48286.441362,свадьба
4697,0,401635.032697,56,среднее,1,женат / замужем,0,F,пенсионер,0,48242.322502,покупка недвижимости
13420,0,401619.633298,63,Среднее,1,гражданский брак,1,F,пенсионер,0,51449.788325,сыграть свадьбу
17823,0,401614.475622,59,среднее,1,женат / замужем,0,F,пенсионер,0,152769.694536,покупка жилья для сдачи
10991,0,401591.828457,56,среднее,1,в разводе,3,F,пенсионер,0,39513.517543,получение дополнительного образования
8369,0,401590.452231,58,среднее,1,женат / замужем,0,F,пенсионер,0,175306.312902,образование


#### 401755 дней стажа \ 365 дней в году = 1100 лет стажа.
И таких данных много. Похоже на техническую ошибку. Стоит рассказать источнику данных о том, что некоторые данные записаны в часах. 

Предположим, что максимальный адекватный стаж в днях - 20000. Напишем функцию, которая будет обрабатывать строки в столбце days_employed и будет считать значения выше 20000 за часы и переводить их в дни. Применим метод apply(), который берёт значения столбца датафрейма и применяет к ним функцию из своего аргумента.

In [8]:
def hours_to_days(row):
    if row > 20000: 
        return row / 24
    return row

data['days_employed'].apply(hours_to_days)

0         8437.673028
1         4024.803754
2         5623.422610
3         4124.747207
4        14177.753002
             ...     
21520     4529.316663
21521    14330.725172
21522     2113.346888
21523     3112.481705
21524     1984.507589
Name: days_employed, Length: 21525, dtype: float64

Days_employed теперь читабелен и почищен, в следующем шаге изменим его тип данных. 

##### Обработка столбца total_income

Займемся следующим столбцом с пропусками - total_income. Посмотрим, в каких рабочих группах встречаются пропуски.

In [9]:
data_nan_income = data[data['total_income'].isna()]
data_nan_income.groupby('income_type')['total_income'].unique()

income_type
госслужащий        [nan]
компаньон          [nan]
пенсионер          [nan]
предприниматель    [nan]
сотрудник          [nan]
Name: total_income, dtype: object

Заменим NaNы на медиану, найденную из доходов определенных рабочих групп из столбца income_type.

In [10]:
#для сотрудников
employers_median = data.loc[(data['income_type'] == 'сотрудник'), 'total_income'].median()
data[data['income_type'] == 'сотрудник'] = data[data['income_type'] == 'сотрудник'].fillna(employers_median)

#для компаньенов
companion_median = data.loc[(data['income_type'] == 'компаньон'), 'total_income'].median()
data[data['income_type'] == 'компаньон'] = data[data['income_type'] == 'компаньон'].fillna(employers_median)

#для пенсионеров
pensioner_median = data.loc[(data['income_type'] == 'пенсионер'), 'total_income'].median()
data[data['income_type'] == 'пенсионер'] = data[data['income_type'] == 'пенсионер'].fillna(employers_median)

#для госслужащих 
governer_median = data.loc[(data['income_type'] == 'госслужащий'), 'total_income'].median()
data[data['income_type'] == 'госслужащий'] = data[data['income_type'] == 'госслужащий'].fillna(employers_median)

#для предпренимателей
entrepreneurs_median = data.loc[(data['income_type'] == 'предприниматель'), 'total_income'].median()
data[data['income_type'] == 'предприниматель'] = data[data['income_type'] == 'предприниматель'].fillna(employers_median)

Проверим количество нулевых значений в таблице.

In [11]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

##### Обработка столбца children

Проверим столбец children, т.к в нем были замечены странные значения.

In [12]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Обнаружились отрицательные значения и аномальная 20-ка. Приведем числа к абсолютным, что-бы исключить минусы и предположим, что 20 - это неверная запись 2 и вернем эти значения на свое место.

In [13]:
data['children'] = abs(data['children'])
data['children'] = data['children'].replace(20, 2)
data['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

### Вывод

Похоже, что пропуски в данных вызваны ошибкой при их обработке или записи. Также тот факт, что большая часть данных записана в часах, может исказить реальную картину, о чем следует сказать источнику данных.

### Замена типа данных

Посмотри, какие типы данных есть в нашей таблице.

In [14]:
data.dtypes

children              int64
days_employed       float64
dob_years             int64
education            object
education_id          int64
family_status        object
family_status_id      int64
gender               object
income_type          object
debt                  int64
total_income        float64
purpose              object
dtype: object

Меня смущает, что дни в столбце days_employed записаны как числа с плавающей точкой. Изменим вещественный тип данных на целочисленный, т.к. это полные сутки. Используем метод astype(), который позволяет четко указывать dtype, который мы хотим иметь в своем DataFrame. Он очень универсален в том, что мы можем попробовать перейти от одного типа к другому.

In [15]:
data['days_employed'] = data['days_employed'].astype('int')

Также можно изменить тип данных у столбца total_income.

In [16]:
data['total_income'] = data['total_income'].astype('int')

### Вывод

Данные бывают разных типов, не всегда походящих для анализа. Для удобства дальнейших расчетов можно изменять типы данных.

### Обработка дубликатов

Проверим, есть ли в таблице дубликаты.

In [17]:
data.duplicated().sum()

54

Рассмотрим их.

In [18]:
data[data.duplicated()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,142594,41,среднее,1,женат / замужем,0,F,сотрудник,0,142594,покупка жилья для семьи
4182,1,142594,34,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,142594,свадьба
4851,0,142594,60,среднее,1,гражданский брак,1,F,пенсионер,0,142594,свадьба
5557,0,142594,58,среднее,1,гражданский брак,1,F,пенсионер,0,142594,сыграть свадьбу
7808,0,142594,57,среднее,1,гражданский брак,1,F,пенсионер,0,142594,на проведение свадьбы
8583,0,142594,58,высшее,0,Не женат / не замужем,4,F,пенсионер,0,142594,дополнительное образование
9238,2,142594,34,среднее,1,женат / замужем,0,F,сотрудник,0,142594,покупка жилья для сдачи
9528,0,142594,66,среднее,1,вдовец / вдова,2,F,пенсионер,0,142594,операции со своей недвижимостью
9627,0,142594,56,среднее,1,женат / замужем,0,F,пенсионер,0,142594,операции со своей недвижимостью
10462,0,142594,62,среднее,1,женат / замужем,0,F,пенсионер,0,142594,покупка коммерческой недвижимости


54 строки с одинаковыми значениями в столбцах days_employed и total_income. Похоже на техническую ошибку, стоит сказать об этом источнику данных.

##### Удаление дубликатов

Т.к. дублированных строк много, стоит выкинуть их из таблицы, чтобы они не искажали реальную картину. Для удаления дубликатов используем метод drop_duplicates() с вызовом метода reset_index(). Тогда создаётся новый DataFrame, где старые индексы превращаются в обычный столбец под названием index, а индексы всех строк снова следуют в естественном порядке.

In [19]:
data = data.drop_duplicates().reset_index(drop = True)
data.duplicated().sum()

0

##### Приведение слов к нижнему регистру

Значения в столбцах education и family_status отличаются по регистру, из-за чего Python не поймет, что это одни и те же данные. Приведем их к нижнему регистру и проверим на наличие дубликатов и аномалий подсчетом уникальных значений.

In [20]:
data['education'] = data['education'].str.lower()
data['education'].value_counts()

среднее                15188
высшее                  5251
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [21]:
data['family_status'] = data['family_status'].str.lower()
data['family_status'].value_counts()

женат / замужем          12344
гражданский брак          4163
не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

### Вывод

Помимо вредного влияния дубликатов на результат, их наличие затягивает процесс анализа. Большое количество повторов неоправданно раздувает размер таблицы, а значит, увеличивает время обработки данных.

### Лемматизация

Посмотрим, какие причины встречаются в столбце с целями получения кредита подсчетом уникальных значений.

In [22]:
data['purpose'].value_counts()

свадьба                                   793
на проведение свадьбы                     773
сыграть свадьбу                           769
операции с недвижимостью                  675
покупка коммерческой недвижимости         662
покупка жилья для сдачи                   652
операции с жильем                         652
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          625
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Свадьба, недвижимость, жилье, образование и автомобиль. С использованием таких слов описывают причину взятия кредита. Создадим функцию для столбца purpose, которая будет принимать строку и преобразовывать слова в ней к их леммам и возвращать название категории. 

In [23]:
def purpose_category(row):
    lemmas = m.lemmatize(row)
    if 'недвижимость' in lemmas:
        return 'недвижимость'
    if 'жилье' in lemmas:
        return 'недвижимость'
    if 'автомобиль' in lemmas:
        return 'автомобиль'
    if 'свадьба' in lemmas:
        return 'свадьба'
    if 'образование' in lemmas:
        return 'образование'

### Вывод

Лемматизация - удобное средство для обработки категориальных типов данных, т.к. аналитики работают не только с числами, но и со словами.

### Категоризация данных

##### Категория целей кредита

Т.к. в столбце с целями получения кредита довольно много причин, описанных разными словами, мы можем категоризировать их по общим словам. Создадим столбец с категориями причин применив метод apply().

In [24]:
data['purpose_category'] = data['purpose'].apply(purpose_category)

##### Категория количества детей

Категоризируем данные количества детей, где значения больше 3х будут считать семью за многодетную.

In [25]:
def children_category(row):
    if row > 3: return '>3'
    if row == 2 or row == 3: return '2-3'
    if row == 1: return '1'
    return '0'

data['children_category'] = data['children'].apply(children_category)

##### Категория уровня дохода

Чтобы ответить на вопрос "есть ли зависимость между уровнем дохода и возвратом кредита в срок?", нам нужно создать категорию с уровнями дохода, которые выведем из значений в таблице total_income.

In [35]:
#def income_category(row):
#    if row < 60000:
#        return 'низкий'
#    if row > 60000 and row < 120000:
#        return 'средний'
#    if row > 120000:
#        return 'высокий'

income_quantiles = data['total_income'].quantile([0, 0.25, 0.5, 0.75, 1])

def income_category(row):
    if row > 20667 and row < 142594:
        return 'низкий'
    if row > 107654 and row < 195751:
        return 'средний'
    if row > 142594 and  row < 2265604:
        return 'высокий'

data['income_category'] = data['total_income'].apply(income_category)    

### Вывод

Чтобы выделить данные по определенному сегменту,  прибегают к категоризации - объединению избранных данных в произвольные группы по заданному критерию. Аналитики пишут функции на Python, чтобы выделить такие группы.

### Шаг 3. Ответьте на вопросы

##### Есть ли зависимость между наличием детей и возвратом кредита в срок?

Для ответа на данный вопрос, создадим сводную таблицу методом pivot_table(), который будет принимать в себя столбец, по которому группируют данные (в этом случае по категории наличия детей), столбец, по значениям которого происходит группировка (есть долг или нет), значения, по которым мы хотим увидеть сводную таблицу (количество детей) и функцию, применяемая к значениям (посчитать количество значений). Далее добавим в эту таблицу соотношение людей вернувших кредит в срок и не вернувших, процент которой покажет, какая группа чаще не возвращает кредит.

In [28]:
data_children_pivot= data.pivot_table(index = ['children_category'], columns = 'debt', values = 'children', aggfunc = 'count')
data_children_pivot['вероятность %'] = data_children_pivot[1] / (data_children_pivot[1] +  data_children_pivot[0])
data_children_pivot.sort_values(by = 'вероятность %', ascending = False)

debt,0,1,вероятность %
children_category,,,
2-3,2229,229,0.093165
1,4411,445,0.091639
>3,46,4,0.080000
0,13044,1063,0.075353


### Вывод

Таблица показывает, что вероятность не вернуть кредит больше у людей с детьми.

##### Есть ли зависимость между семейным положением и возвратом кредита в срок?

Создадим таблицу с соотношением вернувших и невернувших по состоянию семеного пололжения.

In [29]:
data_family_pivot = data.pivot_table(index = ['family_status'], columns = 'debt', values = 'gender', aggfunc = 'count')
data_family_pivot['вероятность %'] = data_family_pivot[1] / (data_family_pivot[1] +  data_family_pivot[0])
data_family_pivot.sort_values(by = 'вероятность %', ascending = False)

debt,0,1,вероятность %
family_status,,,
не женат / не замужем,2536,274,0.097509
гражданский брак,3775,388,0.093202
женат / замужем,11413,931,0.075421
в разводе,1110,85,0.071130
вдовец / вдова,896,63,0.065693


### Вывод

Не женатые и люди в гражданском браке имеют большую вероятность не вернуть кредит в срок. 

##### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Создадим таблицу с соотношением вернувших и невернувших кредит в срок по уровню дохода.

In [30]:
data_income_pivot = data.pivot_table(index = ['income_category'], columns = 'debt', values = 'total_income', aggfunc = 'count')
data_income_pivot['вероятность %'] = data_income_pivot[1] / (data_income_pivot[1] +  data_income_pivot[0])
data_income_pivot.sort_values(by = 'вероятность %', ascending = False)


debt,0,1,вероятность %
income_category,,,
средний,6134,581,0.086523
низкий,8610,776,0.082676
высокий,4985,383,0.071349


### Вывод

Люди со средним и низким доходом чаще не возвращают кредит в срок.

##### Как разные цели кредита влияют на его возврат в срок?

Создадим таблицу с соотношением вернувших и невернувших кредит в срок по целям кредита.

In [31]:
data_purpose_pivot = data.pivot_table(index = ['purpose_category'], columns = 'debt', values = 'gender', aggfunc = 'count')
data_purpose_pivot['вероятность %'] = data_purpose_pivot[1] / (data_purpose_pivot[1] +  data_purpose_pivot[0])
data_purpose_pivot.sort_values(by = 'вероятность %', ascending = False)

debt,0,1,вероятность %
purpose_category,,,
автомобиль,3905,403,0.093547
образование,3644,370,0.092177
свадьба,2149,186,0.079657
недвижимость,10032,782,0.072314


### Вывод

Люди берущие кредит на автомобиль и образование чаще не возвращают кредит в срок.

### Шаг 4. Общий вывод

Человек с детьми, не замужем либо в гражданском браке, с низким или средним уровнем дохода, берущий кредит для операций с автомобилем или на образование - скорее всего будет иметь проблемы с возвратом этого кредита. Такое представление складывается из анализа преобработанной таблицы путем категоризации данных по заданным критериям.